# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 9 2022 12:13PM,252732,22,12092022,FUJIFILM Diosynth Biotechnologies Texas LLC,Executing
1,Dec 9 2022 12:05PM,252730,10,0086237326,ISDIN Corporation,Released
2,Dec 9 2022 12:05PM,252730,10,0086237324,ISDIN Corporation,Released
3,Dec 9 2022 12:05PM,252730,10,0086237325,ISDIN Corporation,Released
4,Dec 9 2022 12:05PM,252730,10,0086237327,ISDIN Corporation,Released
5,Dec 9 2022 12:05PM,252730,10,0086237341,ISDIN Corporation,Released
6,Dec 9 2022 12:05PM,252730,10,0086237346,ISDIN Corporation,Released
7,Dec 9 2022 12:05PM,252730,10,0086237350,ISDIN Corporation,Released
8,Dec 9 2022 12:05PM,252730,10,0086237360,ISDIN Corporation,Released
9,Dec 9 2022 12:05PM,252730,10,0086237366,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
52,252727,Released,1
53,252728,Released,14
54,252729,Released,16
55,252730,Released,11
56,252732,Executing,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
252727,NaN,NaN,1.0
252728,NaN,NaN,14.0
252729,NaN,NaN,16.0
252730,NaN,NaN,11.0
252732,NaN,1.0,NaN


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
252625,0.0,0.0,4.0
252627,0.0,1.0,2.0
252636,0.0,0.0,1.0
252637,0.0,0.0,1.0
252642,0.0,0.0,7.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
252625,0,0,4
252627,0,1,2
252636,0,0,1
252637,0,0,1
252642,0,0,7


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,252625,0,0,4
1,252627,0,1,2
2,252636,0,0,1
3,252637,0,0,1
4,252642,0,0,7


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,252625,,,4
1,252627,,1,2
2,252636,,,1
3,252637,,,1
4,252642,,,7


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 9 2022 12:13PM,252732,22,FUJIFILM Diosynth Biotechnologies Texas LLC
1,Dec 9 2022 12:05PM,252730,10,ISDIN Corporation
12,Dec 9 2022 12:05PM,252729,10,ISDIN Corporation
28,Dec 9 2022 12:04PM,252728,19,"AdvaGen Pharma, Ltd"
42,Dec 9 2022 11:59AM,252727,19,ACG North America LLC
43,Dec 9 2022 11:57AM,252726,10,ISDIN Corporation
51,Dec 9 2022 11:54AM,252725,10,Bio-PRF
52,Dec 9 2022 11:54AM,252724,10,Bio-PRF
53,Dec 9 2022 11:52AM,252723,19,"GUSA Granules USA, Inc."
54,Dec 9 2022 11:43AM,252721,10,"Nextsource Biotechnology, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Dec 9 2022 12:13PM,252732,22,FUJIFILM Diosynth Biotechnologies Texas LLC,,1,
1,Dec 9 2022 12:05PM,252730,10,ISDIN Corporation,,,11
2,Dec 9 2022 12:05PM,252729,10,ISDIN Corporation,,,16
3,Dec 9 2022 12:04PM,252728,19,"AdvaGen Pharma, Ltd",,,14
4,Dec 9 2022 11:59AM,252727,19,ACG North America LLC,,,1
5,Dec 9 2022 11:57AM,252726,10,ISDIN Corporation,,,8
6,Dec 9 2022 11:54AM,252725,10,Bio-PRF,,,1
7,Dec 9 2022 11:54AM,252724,10,Bio-PRF,,,1
8,Dec 9 2022 11:52AM,252723,19,"GUSA Granules USA, Inc.",,,1
9,Dec 9 2022 11:43AM,252721,10,"Nextsource Biotechnology, LLC",,,3


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 9 2022 12:13PM,252732,22,FUJIFILM Diosynth Biotechnologies Texas LLC,,1,
1,Dec 9 2022 12:05PM,252730,10,ISDIN Corporation,11,,
2,Dec 9 2022 12:05PM,252729,10,ISDIN Corporation,16,,
3,Dec 9 2022 12:04PM,252728,19,"AdvaGen Pharma, Ltd",14,,
4,Dec 9 2022 11:59AM,252727,19,ACG North America LLC,1,,
5,Dec 9 2022 11:57AM,252726,10,ISDIN Corporation,8,,
6,Dec 9 2022 11:54AM,252725,10,Bio-PRF,1,,
7,Dec 9 2022 11:54AM,252724,10,Bio-PRF,1,,
8,Dec 9 2022 11:52AM,252723,19,"GUSA Granules USA, Inc.",1,,
9,Dec 9 2022 11:43AM,252721,10,"Nextsource Biotechnology, LLC",3,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 9 2022 12:13PM,252732,22,FUJIFILM Diosynth Biotechnologies Texas LLC,,1,
1,Dec 9 2022 12:05PM,252730,10,ISDIN Corporation,11,,
2,Dec 9 2022 12:05PM,252729,10,ISDIN Corporation,16,,
3,Dec 9 2022 12:04PM,252728,19,"AdvaGen Pharma, Ltd",14,,
4,Dec 9 2022 11:59AM,252727,19,ACG North America LLC,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 9 2022 12:13PM,252732,22,FUJIFILM Diosynth Biotechnologies Texas LLC,NaN,1.0,NaN
1,Dec 9 2022 12:05PM,252730,10,ISDIN Corporation,11.0,NaN,NaN
2,Dec 9 2022 12:05PM,252729,10,ISDIN Corporation,16.0,NaN,NaN
3,Dec 9 2022 12:04PM,252728,19,"AdvaGen Pharma, Ltd",14.0,NaN,NaN
4,Dec 9 2022 11:59AM,252727,19,ACG North America LLC,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 9 2022 12:13PM,252732,22,FUJIFILM Diosynth Biotechnologies Texas LLC,0.0,1.0,0.0
1,Dec 9 2022 12:05PM,252730,10,ISDIN Corporation,11.0,0.0,0.0
2,Dec 9 2022 12:05PM,252729,10,ISDIN Corporation,16.0,0.0,0.0
3,Dec 9 2022 12:04PM,252728,19,"AdvaGen Pharma, Ltd",14.0,0.0,0.0
4,Dec 9 2022 11:59AM,252727,19,ACG North America LLC,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5811872,155.0,5.0,1.0
15,758091,14.0,0.0,0.0
19,1768897,21.0,0.0,0.0
20,252653,0.0,1.0,0.0
21,4800639,19.0,0.0,0.0
22,252732,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5811872,155.0,5.0,1.0
1,15,758091,14.0,0.0,0.0
2,19,1768897,21.0,0.0,0.0
3,20,252653,0.0,1.0,0.0
4,21,4800639,19.0,0.0,0.0
5,22,252732,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,155.0,5.0,1.0
1,15,14.0,0.0,0.0
2,19,21.0,0.0,0.0
3,20,0.0,1.0,0.0
4,21,19.0,0.0,0.0
5,22,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,155.0
1,15,Released,14.0
2,19,Released,21.0
3,20,Released,0.0
4,21,Released,19.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,19,20,21,22
Status,,,,,,
Completed,1.0,0.0,0.0,0.0,0.0,0.0
Executing,5.0,0.0,0.0,1.0,0.0,1.0
Released,155.0,14.0,21.0,0.0,19.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,19,20,21,22
0,Completed,1.0,0.0,0.0,0.0,0.0,0.0
1,Executing,5.0,0.0,0.0,1.0,0.0,1.0
2,Released,155.0,14.0,21.0,0.0,19.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,19,20,21,22
0,Completed,1.0,0.0,0.0,0.0,0.0,0.0
1,Executing,5.0,0.0,0.0,1.0,0.0,1.0
2,Released,155.0,14.0,21.0,0.0,19.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()